In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"  # Or "jax" or "torch"!

import tensorflow as tf

import tensorflow_datasets as tfds


In [3]:
pip install keras_cv


   ---------------------------------------- 0.0/650.7 kB ? eta -:--:--
   --------------------------------------- 650.7/650.7 kB 12.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/950.8 kB ? eta -:--:--
   --------------------------------------- 950.8/950.8 kB 11.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import keras_cv

d:\a27_YEARS_OLD\deep_learning\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import keras

In [6]:
# Create a preprocessing pipeline with augmentations
BATCH_SIZE = 16
NUM_CLASSES = 3
augmenter = keras_cv.layers.Augmenter(
    [
        keras_cv.layers.RandomFlip(),
        keras_cv.layers.RandAugment(value_range=(0, 255)),
        keras_cv.layers.CutMix(),
    ],
)

In [7]:
def preprocess_data(images, labels, augment=False):
    labels = tf.one_hot(labels, NUM_CLASSES)
    inputs = {"images": images, "labels": labels}
    outputs = inputs
    if augment:
        outputs = augmenter(outputs)
    return outputs['images'], outputs['labels']

In [ ]:
train_dataset, test_dataset = tfds.load(
    'rock_paper_scissors',
    as_supervised=True,
    split=['train', 'test'],
    data_dir=os.getcwd())

Dl Completed...: 100%|██████████| 2/2 [00:24<00:00, 12.29s/ url]


Dataset rock_paper_scissors downloaded and prepared to C:\Users\lenovo\tensorflow_datasets\rock_paper_scissors\3.0.0. Subsequent calls will reuse this data.


In [10]:
train_dataset = train_dataset.batch(BATCH_SIZE).map(
    lambda x, y: preprocess_data(x, y, augment=True),
        num_parallel_calls=tf.data.AUTOTUNE).prefetch(
            tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).map(
    preprocess_data, num_parallel_calls=tf.data.AUTOTUNE).prefetch(
        tf.data.AUTOTUNE)

In [11]:
# Create a model using a pretrained backbone
backbone = keras_cv.models.EfficientNetV2Backbone.from_preset(
    "efficientnetv2_b0_imagenet"
)

100%|██████████| 1.79k/1.79k [00:00<00:00, 911kB/s]


100%|██████████| 23.1M/23.1M [00:04<00:00, 5.58MB/s]


In [12]:
model = keras_cv.models.ImageClassifier(
    backbone=backbone,
    num_classes=NUM_CLASSES,
    activation="softmax",
)

In [13]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    metrics=['accuracy']
)

In [15]:
# Train your model
model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=2,
)

Epoch 1/2


ValueError: Creating variables on a non-first call to a function decorated with tf.function.